In [28]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import re

In [29]:
# ==== 1. Load model dan tokenizer ====
model_path = "./sentiment_model"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [30]:
# ==== 2. Mapping label angka ke string ====
label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "others"
}

In [31]:
# ==== 3. Fungsi preprocessing untuk input user ====
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [32]:
# ==== 4. Fungsi prediksi emosi dari inputan user ====
def predict_emotion(text):
    clean_text_input = clean_text(text)
    inputs = tokenizer(clean_text_input, return_tensors="pt", truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    label_id = torch.argmax(logits, dim=1).item()
    probs = torch.softmax(logits, dim=1).squeeze().tolist()
    return label_map[label_id], probs

In [33]:
# ==== 5. Load dataset film dan musik ====
movies_df = pd.read_csv("movies.csv")
music_df = pd.read_csv("music.csv")

In [34]:
# ==== 6. Pastikan kolom label dalam lowercase ====
movies_df['mapped_label'] = movies_df['label'].str.lower()
music_df['mapped_label'] = music_df['mapped_label'].str.lower()

In [35]:
# ==== 7. Fungsi rekomendasi film ====
def recommend_movies(emotion_label, top_n=3):
    filtered = movies_df[movies_df['mapped_label'] == emotion_label]
    if len(filtered) >= top_n:
        sampled = filtered.sample(n=top_n, random_state=None)
    else:
        sampled = filtered
    return sampled[['title', 'overview']].to_dict(orient='records')

In [36]:
# ==== 8. Fungsi rekomendasi musik  ====
def recommend_music(emotion_label, top_n=3):
    filtered = music_df[music_df['mapped_label'] == emotion_label]
    if len(filtered) >= top_n:
        sampled = filtered.sample(n=top_n, random_state=None)  # acak setiap panggilan
    else:
        sampled = filtered
    return sampled[['track', 'artist', 'combined_seeds']].to_dict(orient='records')

In [37]:
# ==== 9. Fungsi utama untuk menghasilkan rekomendasi ====
def get_recommendations(user_input):
    emotion, probs = predict_emotion(user_input)
    movies = recommend_movies(emotion)
    music = recommend_music(emotion)
    return {
        "input": user_input,
        "predicted_emotion": emotion,
        "movies": movies,
        "music": music
    }

In [39]:
# ==== 10. Contoh pemakaian ====
if __name__ == "__main__":
    user_text = input("Input your feelings: ")
    result = get_recommendations(user_text)

    print("Detected Mood:", result["predicted_emotion"])
    print("\nFilm Recommendation:")
    for movie in result["movies"]:
        print(f"- {movie['title']}: {movie['overview'][:100]}...")

    print("\nMusic Recommendation:")
    for track in result["music"]:
        print(f"- {track['track']} by {track['artist']} ({track['combined_seeds']})")

Detected Mood: sadness

Film Recommendation:
- A Nutcracker Christmas: When her niece is cast in The Philadelphia Ballet’s production of the Nutcracker, a jaded ex-balleri...
- The Marva Collins Story: Cicely Tyson was Emmy-nominated as Outstanding Actress for her portrait of a Chicago schoolteacher w...
- The Red Badge of Courage: Truncated adaptation of Stephen Crane's novel about a Civil War Union soldier who stuggles to find t...

Music Recommendation:
- No Need To Worry by Yeasayer (epic sad)
- I Like Your Old Stuff Better Than Your New Stuff by Regurgitator (silly)
- Bruce's Song by Monty Python (serious)
